In [1]:
from helper_lstm import *
import pandas as pd

Using TensorFlow backend.


In [2]:
country='India'
# Setting up an early stop
earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=80,  verbose=1, mode='min')
#earlystop = EarlyStopping(monitor='mean_squared_error', min_delta=0.0001, patience=80,  verbose=1, mode='min')
callbacks_list = [earlystop]
# Train Val Test Split
series=get_data(country)
#train,val,test=get_train_test(series)
#print(series)

C:\Users\ase072\Documents\Research\ratnabali\code_github\helper_lstm.py:53: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  dateparse = lambda dates: [pd.datetime.strptime(d, '%d-%m-%Y') for d in dates]


In [3]:
@use_named_args(dimensions=dimensions)
def fitness(learning_rate,num_of_layers,num_hidden_nodes,activation, batch_size,dropout,timesteps=1):
      
    train,val,test,sc=get_train_test(series)
    #named blackbox becuase it represents the structure
    X_train = []
    Y_train = []
    X_val = []
    Y_val = []
  
    # Loop for training data
    for i in range(timesteps,train.shape[0]):
        X_train.append(train[i-timesteps:i])
        Y_train.append(train[i][0])
    X_train,Y_train = np.array(X_train),np.array(Y_train)
  
    # Loop for val data
    for i in range(timesteps,val.shape[0]):
        X_val.append(val[i-timesteps:i])
        Y_val.append(val[i][0])
    X_val,Y_val = np.array(X_val),np.array(Y_val)
    
    print('Creating Model')
    model = create_model(X_train,learning_rate=learning_rate,
                         num_of_layers=num_of_layers,
                         num_hidden_nodes=num_hidden_nodes,
                         activation=activation,
                         dropout=dropout
                        )
     
    blackbox = model.fit(X_train,Y_train,epochs = 5000,batch_size = batch_size,validation_data = (X_val, Y_val),verbose = 0,callbacks=callbacks_list)
    
    accuracy = blackbox.history['val_loss'][-1]
    #accuracy = blackbox.history['mean_squared_error'][-1]
    # Print the classification accuracy.
    print(accuracy)
    #print("Accuracy: {0:.2%}".format(accuracy))
    print()


    # Delete the Keras model with these hyper-parameters from memory.
    del model
    
    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    #tensorflow.reset_default_graph()
    
    return accuracy

In [4]:
import pickle

f = open(country+'_lstm2.pckl', 'wb')
pickle.dump(gp_result, f)
f.close()

NameError: name 'gp_result' is not defined

In [ ]:
# Evaluating the model
def evaluate_model(model,test,timesteps):
    X_test = []
    Y_test = []

    # Loop for testing data
    for i in range(timesteps,test.shape[0]):
        X_test.append(test[i-timesteps:i])
        Y_test.append(test[i][0])
    X_test,Y_test = np.array(X_test),np.array(Y_test)
    #print(X_test.shape,Y_test.shape)
  
    # Prediction Time !!!!
    Y_hat = model.predict(X_test)
    mse = mean_squared_error(Y_test,Y_hat)
    rmse = sqrt(mse)
    r = r2_score(Y_test,Y_hat)
    return mse, rmse, r, Y_test, Y_hat
# Plotting the predictions
def plot_data(Y_test,Y_hat):
    plt.plot(Y_test,c = 'r')
    plt.plot(Y_hat,c = 'y')
    plt.xlabel('Day')
    plt.ylabel('Price')
    plt.title('Stock Prediction Graph using Multivariate-LSTM model')
    plt.legend(['Actual','Predicted'],loc = 'lower right')
    plt.show()
# Plotting the training errors
def plot_error(train_loss,val_loss):
    plt.plot(train_loss,c = 'r')
    plt.plot(val_loss,c = 'b')
    plt.ylabel('Loss')
    plt.legend(['train','val'],loc = 'upper right')
    plt.show()

In [ ]:
def get_original_train_test(series):
    # Train Val Test Split
    train_start = dt.date(2020,1,22)
    train_end = dt.date(2020,3,18)
    train_data = series.loc[train_start:train_end]

    val_start = dt.date(2020,3,18)
    val_end = dt.date(2020,3,25)
    val_data = series.loc[val_start:val_end]

    test_start = dt.date(2020,3,18)
    test_end = dt.date(2020,3,25)
    test_data = series.loc[test_start:test_end]

#print(train_data.shape,val_data.shape,test_data.shape)
    # Normalisation
    sc = MinMaxScaler()
    train = sc.fit_transform(train_data)
    val = sc.transform(val_data)
    test = sc.transform(test_data)
    return train_data,val_data,test_data,sc

In [ ]:
timesteps=5
train,val,test,sc1=get_train_test(series)
#train,val,test,sc=get_original_train_test(series)
    #named blackbox becuase it represents the structure
train_data,val_data,test_data,sc=get_original_train_test(series)
X_train = []
Y_train = []
X_val = []
Y_val = []
  
    # Loop for training data
for i in range(timesteps,train.shape[0]):
    X_train.append(train[i-timesteps:i])
    Y_train.append(train[i][0])
X_train,Y_train = np.array(X_train),np.array(Y_train)
  
    # Loop for val data
for i in range(timesteps,val.shape[0]):
    X_val.append(val[i-timesteps:i])
    Y_val.append(val[i][0])
X_val,Y_val = np.array(X_val),np.array(Y_val)
#model = create_model(X_train, gp_result.x[0],gp_result.x[1],10,gp_result.x[3],gp_result.x[4])
model = create_model1(X_train, 0.00028226662453909794,3,170,'relu',0)
#model = create_model1(X_train, 0.0001,20,200,'relu',0)
history=model.fit(X_train,Y_train, epochs=500,callbacks=callbacks_list,validation_data = (X_val, Y_val),batch_size=6)
mse, rmse, r2_value,true,predicted = evaluate_model(model,test,timesteps)
print('MSE = {}'.format(mse))
print('RMSE = {}'.format(rmse))
print('R-Squared Score = {}'.format(r2_value))
plot_data(true,predicted)
model.evaluate(X_val,Y_val)

In [ ]:
print(gp_result.x[0],gp_result.x[1],gp_result.x[2],gp_result.x[3],gp_result.x[4])

In [ ]:
val2=test.copy()
val3=test.copy()
val2[:,0]=true
val3[:,0]=predicted.ravel()

In [ ]:
val_original=sc.inverse_transform(val2)
val_prediction=sc.inverse_transform(val3)
plot_data(val_original[:,0],val_prediction[:,0])

In [ ]:
train_data,val_data,test_data,sc=get_original_train_test(series)
plt.plot(train_data,c = 'r')
plt.plot(val_prediction[:,0],c = 'y')
plt.xlabel('Day')
plt.ylabel('Price')
plt.title('Stock Prediction Graph using Multivariate-LSTM model')
plt.legend(['Actual','Predicted'],loc = 'lower right')
plt.show()

In [ ]:
val2[:,0].shape

In [ ]:
true.shape

In [ ]:
val.shape

In [ ]:
series